In [3]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'Home.settings')
django.setup()
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [4]:
from basemodels.models import FikstureModelData

In [5]:
#verileri oku
,
import pandas as pd

data = FikstureModelData.objects.all().values()
data_list = list(data)
df = pd.DataFrame(data_list)
df

,id,data_id,tarih,data,count,isprogress,created_at,updated_at
0,1,20241021,2024-10-21,"[{'tournament': {'name': 'Premier League', 'sl...",333,False,2024-10-21 11:18:48.967709+00:00,2024-10-21 11:18:48.967709+00:00
1,6,20241020,2024-10-20,"[{'tournament': {'name': 'Premier League', 'sl...",744,False,2024-10-21 12:03:20.963828+00:00,2024-10-21 12:03:20.963828+00:00
2,13,20241019,2024-10-19,"[{'tournament': {'name': 'Premier League', 'sl...",826,False,2024-10-21 12:24:06.417397+00:00,2024-10-21 12:24:06.417397+00:00
3,16,20241018,2024-10-18,"[{'tournament': {'name': 'Premier League', 'sl...",411,False,2024-10-21 13:04:53.455994+00:00,2024-10-21 13:04:53.455994+00:00
4,20,20241017,2024-10-17,[{'tournament': {'name': 'Brasileirão Série A'...,83,False,2024-10-21 13:10:11.208836+00:00,2024-10-21 13:10:11.208836+00:00
5,21,20241016,2024-10-16,"[{'tournament': {'name': 'UEFA Nations League,...",120,False,2024-10-21 13:14:11.457338+00:00,2024-10-21 13:14:11.457338+00:00
6,22,20241001,2024-10-01,[{'tournament': {'name': 'UEFA Champions Leagu...,176,False,2024-10-21 13:27:48.695696+00:00,2024-10-21 13:27:48.695696+00:00
7,23,20241002,2024-10-02,[{'tournament': {'name': 'UEFA Champions Leagu...,206,False,2024-10-21 13:27:53.223685+00:00,2024-10-21 13:27:53.223685+00:00
8,24,20241003,2024-10-03,[{'tournament': {'name': 'UEFA Champions Leagu...,203,False,2024-10-21 13:27:56.535289+00:00,2024-10-21 13:27:56.535289+00:00
9,25,20241004,2024-10-04,"[{'tournament': {'name': 'Premier League', 'sl...",422,False,2024-10-21 13:28:00.910519+00:00,2024-10-21 13:28:00.910519+00:00


In [9]:
from basemodels.models import Tournament,Season

# Modelleri import edin
import pandas as pd

class SofascoreAPI:
    def __init__(self):
        pass
      
        
    def fetch_data(self):
        
        data = FikstureModelData.objects.all().values()
        data_list = list(data)
        df = pd.DataFrame(data_list)
        df_exploded = df.explode('data')
        expanded_data = pd.json_normalize(df_exploded['data'])
        expanded_data['tarih'] = list(df_exploded['tarih'])
        veri = expanded_data.copy()
        veri.columns = [item.replace(".", "_") for item in veri.columns]

        return veri
        

    def tournament_data(self):

            verim = self.fetch_data()
            """Eksik sütunları kontrol edip, varsa 0 ile doldurma ve veriyi sonlandırma."""
            if verim is None:
                raise ValueError("Veri işlenmedi. Önce 'process_data()' metodunu çalıştırın.")
            
            # Aynı uniqueTournament_id'ye sahip olanları tekrar kontrol et
            verim = verim.drop_duplicates("tournament_uniqueTournament_id")
            
            # İlgili sütunlar
            cols = [
                "tournament_uniqueTournament_id", "tournament_name", "tournament_slug", "tournament_category_name",
                "tournament_category_slug", "tournament_uniqueTournament_category_id", 
                "tournament_uniqueTournament_hasEventPlayerStatistics", "tournament_id", "tournament_isGroup", 
                "tournament_uniqueTournament_hasPerformanceGraphFeature"
            ]
            
            # Eksik sütunları 0 ile doldurma
            for col in cols:
                if col not in verim.columns:
                    verim.loc[:, col] = 0
    
            # Son olarak sadece belirtilen sütunlar kalsın
            verim = verim[cols]

            for _, row in verim.iterrows():
                tournament_id = row['tournament_uniqueTournament_id']
                
                # NaN değerlerini kontrol edin ve NaN olan satırları atlayın
                if pd.isna(tournament_id):
                    print(f"Geçersiz tournament_uniqueTournament_id: {tournament_id}, atlanıyor.")
                    continue  # NaN olan satırı atla
        
                # Modelde bu id ile kayıt olup olmadığını kontrol et
                if not Tournament.objects.filter(tournament_uniqueTournament_id=tournament_id).exists():
                    # Kayıt yoksa yeni kayıt oluştur
                    Tournament.objects.create(
                        tournament_uniqueTournament_id=row['tournament_uniqueTournament_id'],
                        tournament_name=row['tournament_name'],
                        tournament_slug=row['tournament_slug'],
                        tournament_category_name=row['tournament_category_name'],
                        tournament_category_slug=row['tournament_category_slug'],
                        tournament_uniqueTournament_category_id=row['tournament_uniqueTournament_category_id'],
                        tournament_uniqueTournament_hasEventPlayerStatistics=row['tournament_uniqueTournament_hasEventPlayerStatistics'],
                        tournament_id=row['tournament_id'],
                        tournament_isGroup=row['tournament_isGroup'] if pd.notna(row['tournament_isGroup']) else False,
                        tournament_uniqueTournament_hasPerformanceGraphFeature=row['tournament_uniqueTournament_hasPerformanceGraphFeature'] if pd.notna(row['tournament_uniqueTournament_hasPerformanceGraphFeature']) else False
                    )
                    print(f"Tournament ID {tournament_id} başarıyla eklendi.")
                else:
                    print(f"Tournament ID {tournament_id} zaten mevcut, eklenmedi.")



    
    def season_data(self):
        """Season verileri için eksik sütunları kontrol edip, 0 ile doldurma işlemi."""
        verim = self.fetch_data()
        
        if verim is None:
            raise ValueError("Veri işlenmedi. Önce 'process_data()' metodunu çalıştırın.")
        
        # Aynı season_id'ye sahip olanları tekrar kontrol et
        verim = verim.drop_duplicates("season_id")
        
        # Sezonla ilgili sütunlar
        season_cols = ["season_id", "season_name", "season_year","tournament_uniqueTournament_id"]

        # Eksik sütunları 0 ile doldurma
        for col in season_cols:
            if col not in verim.columns:
                verim.loc[:, col] = 0

        # Son olarak sadece season ile ilgili sütunları döndür
        verim = verim[season_cols]

        for _, row in verim.iterrows():
            season_id = row['season_id']
            tournament_id = row.get('tournament_uniqueTournament_id')  # İlgili turnuvayı al
    
            # NaN değerlerini kontrol edin ve geçersiz veriyi atlayın
            if pd.isna(season_id) or pd.isna(tournament_id):
                print(f"Geçersiz season_id veya tournament_id: Season ID={season_id}, Tournament ID={tournament_id}. Atlanıyor.")
                continue
    
            # Modelde bu id ile kayıt olup olmadığını kontrol et
            if not Season.objects.filter(season_id=season_id).exists():
                # Turnuva olup olmadığını kontrol et ve turnuva ile sezonu ilişkilendir
                tournament_instance = Tournament.objects.filter(tournament_uniqueTournament_id=tournament_id).first()
                
                if tournament_instance:
                    # Kayıt yoksa yeni sezon kaydı oluştur
                    Season.objects.create(
                        season_id=row['season_id'],
                        season_name=row['season_name'],
                        season_year=row['season_year'],
                        tournament=tournament_instance  # Turnuva ile ilişkilendir
                    )
                    print(f"Season ID {season_id} başarıyla eklendi.")
                else:
                    print(f"Tournament ID {tournament_id} bulunamadı, Season ID {season_id} eklenemedi.")
            else:
                print(f"Season ID {season_id} zaten mevcut, eklenmedi.")


api = SofascoreAPI()  
api.tournament_data() 
api.season_data()

Tournament ID 17 zaten mevcut, eklenmedi.
Tournament ID 8 zaten mevcut, eklenmedi.
Tournament ID 23 zaten mevcut, eklenmedi.
Tournament ID 35 zaten mevcut, eklenmedi.
Tournament ID 34 zaten mevcut, eklenmedi.
Tournament ID 37 zaten mevcut, eklenmedi.
Tournament ID 325 zaten mevcut, eklenmedi.
Tournament ID 52 zaten mevcut, eklenmedi.
Tournament ID 38 zaten mevcut, eklenmedi.
Tournament ID 185 zaten mevcut, eklenmedi.
Tournament ID 2132 zaten mevcut, eklenmedi.
Tournament ID 155 zaten mevcut, eklenmedi.
Tournament ID 11621 zaten mevcut, eklenmedi.
Tournament ID 170 zaten mevcut, eklenmedi.
Tournament ID 210 zaten mevcut, eklenmedi.
Tournament ID 18 zaten mevcut, eklenmedi.
Tournament ID 54 zaten mevcut, eklenmedi.
Tournament ID 53 zaten mevcut, eklenmedi.
Tournament ID 182 zaten mevcut, eklenmedi.
Tournament ID 390 zaten mevcut, eklenmedi.
Tournament ID 131 zaten mevcut, eklenmedi.
Tournament ID 98 zaten mevcut, eklenmedi.
Tournament ID 45 zaten mevcut, eklenmedi.
Tournament ID 266 zate

In [10]:
data = api.fetch_data()
data

,customId,winnerCode,hasGlobalHighlights,hasEventPlayerStatistics,hasEventPlayerHeatMap,detailId,crowdsourcingDataDisplayEnabled,id,startTimestamp,slug,...,awayTeam_gender,homeScore_aggregated,awayScore_aggregated,homeScore_penalties,awayScore_penalties,coverage,isAwarded,homeScore_overtime,awayScore_overtime,tarih
0,dsr,2.0,True,True,True,1.0,False,12437039,1729429200,manchester-city-wolverhampton,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-21
1,NsU,1.0,True,True,True,1.0,False,12437032,1729438200,liverpool-chelsea,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-21
2,hso,NaN,False,NaN,NaN,1.0,False,12437036,1729537200,nottingham-forest-crystal-palace,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-21
3,tgbsBgb,1.0,True,True,True,1.0,False,12437847,1729425600,mallorca-rayo-vallecano,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-21
4,LgbsVgb,1.0,True,True,True,1.0,False,12437835,1729433700,leganes-atletico-madrid,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5468,fYbspYb,1.0,False,True,True,NaN,False,11362470,1696080600,haverfordwest-county-afc-aberystwyth-town,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-09-30
5469,tYbsfAo,1.0,False,True,NaN,NaN,False,11362472,1696080600,bala-town-newtown-afc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-09-30
5470,wYbsMip,1.0,False,True,NaN,NaN,False,11362467,1696080600,pontypridd-united-the-new-saints,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-09-30
5471,mYbsKdW,1.0,False,True,True,NaN,False,11362471,1696090500,penybont-connahs-quay-nomads,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-09-30


In [12]:
list(data.columns)

['customId',
 'winnerCode',
 'hasGlobalHighlights',
 'hasEventPlayerStatistics',
 'hasEventPlayerHeatMap',
 'detailId',
 'crowdsourcingDataDisplayEnabled',
 'id',
 'startTimestamp',
 'slug',
 'finalResultOnly',
 'feedLocked',
 'isEditor',
 'tournament_name',
 'tournament_slug',
 'tournament_category_name',
 'tournament_category_slug',
 'tournament_category_sport_name',
 'tournament_category_sport_slug',
 'tournament_category_sport_id',
 'tournament_category_id',
 'tournament_category_country_alpha2',
 'tournament_category_country_alpha3',
 'tournament_category_country_name',
 'tournament_category_country_slug',
 'tournament_category_flag',
 'tournament_category_alpha2',
 'tournament_uniqueTournament_name',
 'tournament_uniqueTournament_slug',
 'tournament_uniqueTournament_category_name',
 'tournament_uniqueTournament_category_slug',
 'tournament_uniqueTournament_category_sport_name',
 'tournament_uniqueTournament_category_sport_slug',
 'tournament_uniqueTournament_category_sport_id',
 

In [ ]:
class Team(models.Model):
    team_id = models.IntegerField() 
    team_name = models.CharField(max_length=255)
    team_slug = models.SlugField(max_length=255)
    team_shortName = models.CharField(max_length=100)
    team_nameCode = models.CharField(max_length=10)
    team_national = models.BooleanField(default=False)

  
    def __str__(self):
        return self.team_name

In [18]:
data = api.fetch_data()
homecols = [
            "homeTeam_id", "homeTeam_name", "homeTeam_slug", "homeTeam_shortName",
            "homeTeam_nameCode", "homeTeam_national","tournament_uniqueTournament_id","season_id"
        ]

awaycols = [
            "awayTeam_id", "awayTeam_name", "awayTeam_slug", "awayTeam_shortName",
            "awayTeam_nameCode", "awayTeam_national","tournament_uniqueTournament_id","season_id"
        ]

homedata = data[homecols]
homedata.columns = ["Team_id", "Team_name", "Team_slug", "Team_shortName",
                               "Team_nameCode", "Team_national","tournament_uniqueTournament_id","season_id"]
awaydata = data[awaycols]
awaydata.columns  = ["Team_id", "Team_name", "Team_slug", "Team_shortName",
                               "Team_nameCode", "Team_national","tournament_uniqueTournament_id","season_id"]

teamdata = pd.concat([homedata,awaydata])
teamdata = teamdata.drop_duplicates(subset=["Team_id", "tournament_uniqueTournament_id", "season_id"])
teamdata

,Team_id,Team_name,Team_slug,Team_shortName,Team_nameCode,Team_national,tournament_uniqueTournament_id,season_id
0,3,Wolverhampton,wolverhampton,Wolves,WOL,False,17,61627
1,44,Liverpool,liverpool,Liverpool,LIV,False,17,61627
2,14,Nottingham Forest,nottingham-forest,Forest,NFO,False,17,61627
3,2826,Mallorca,mallorca,Mallorca,MLL,False,8,61643
4,2836,Atlético Madrid,atletico-madrid,Atl. Madrid,ATM,False,8,61643
...,...,...,...,...,...,...,...,...
5468,4905,Aberystwyth Town,aberystwyth-town,Aberystwyth,ABE,False,254,52310
5469,36255,Bala Town,bala-town,Bala Town,BAT,False,254,52310
5470,37937,Pontypridd United,pontypridd-united,Pontypridd United,PPT,False,254,52310
5471,115185,Penybont,penybont,Penybont,PEN,False,254,52310


In [21]:
datam = teamdata[teamdata["season_id"]==61643]
datam

,Team_id,Team_name,Team_slug,Team_shortName,Team_nameCode,Team_national,tournament_uniqueTournament_id,season_id
3,2826,Mallorca,mallorca,Mallorca,MLL,False,8,61643
4,2836,Atlético Madrid,atletico-madrid,Atl. Madrid,ATM,False,8,61643
5,2819,Villarreal,villarreal,Villarreal,VIL,False,8,61643
6,2817,Barcelona,barcelona,Barcelona,FCB,False,8,61643
7,2828,Valencia,valencia,Valencia,VCF,False,8,61643
341,2825,Athletic Club,athletic-club,Athletic Club,ATH,False,8,61643
342,2820,Osasuna,osasuna,Osasuna,OSA,False,8,61643
343,24264,Girona FC,girona-fc,Girona,GIR,False,8,61643
344,2821,Celta Vigo,celta-vigo,Celta,RCC,False,8,61643
1085,2885,Deportivo Alavés,deportivo-alaves,Alavés,ALA,False,8,61643


In [7]:
from basemodels.models import Tournament, Season, Team
import pandas as pd

class SofascoreAPI:
    def __init__(self):
        pass

    def fetch_data(self):
        data = FikstureModelData.objects.all().values()
        data_list = list(data)
        df = pd.DataFrame(data_list)
        df_exploded = df.explode('data')
        expanded_data = pd.json_normalize(df_exploded['data'])
        expanded_data['tarih'] = list(df_exploded['tarih'])
        veri = expanded_data.copy()
        veri.columns = [item.replace(".", "_") for item in veri.columns]
        return veri

    def tournament_data(self):
        verim = self.fetch_data()
        if verim is None:
            raise ValueError("Veri işlenmedi. Önce 'process_data()' metodunu çalıştırın.")
        
        # Aynı uniqueTournament_id'ye sahip olanları tekrar kontrol et
        verim = verim.drop_duplicates("tournament_uniqueTournament_id")
        
        cols = [
            "tournament_uniqueTournament_id", "tournament_name", "tournament_slug", "tournament_category_name",
            "tournament_category_slug", "tournament_uniqueTournament_category_id", 
            "tournament_uniqueTournament_hasEventPlayerStatistics", "tournament_id", "tournament_isGroup", 
            "tournament_uniqueTournament_hasPerformanceGraphFeature"
        ]
        
        for col in cols:
            if col not in verim.columns:
                verim.loc[:, col] = 0

        verim = verim[cols]

        for _, row in verim.iterrows():
            tournament_id = row['tournament_uniqueTournament_id']
            
            if pd.isna(tournament_id):
                print(f"Geçersiz tournament_uniqueTournament_id: {tournament_id}, atlanıyor.")
                continue
        
            if not Tournament.objects.filter(tournament_uniqueTournament_id=tournament_id).exists():
                Tournament.objects.create(
                    tournament_uniqueTournament_id=row['tournament_uniqueTournament_id'],
                    tournament_name=row['tournament_name'],
                    tournament_slug=row['tournament_slug'],
                    tournament_category_name=row['tournament_category_name'],
                    tournament_category_slug=row['tournament_category_slug'],
                    tournament_uniqueTournament_category_id=row['tournament_uniqueTournament_category_id'],
                    tournament_uniqueTournament_hasEventPlayerStatistics=row['tournament_uniqueTournament_hasEventPlayerStatistics'],
                    tournament_id=row['tournament_id'],
                    tournament_isGroup=row['tournament_isGroup'] if pd.notna(row['tournament_isGroup']) else False,
                    tournament_uniqueTournament_hasPerformanceGraphFeature=row['tournament_uniqueTournament_hasPerformanceGraphFeature'] if pd.notna(row['tournament_uniqueTournament_hasPerformanceGraphFeature']) else False
                )
                print(f"Tournament ID {tournament_id} başarıyla eklendi.")
            else:
                print(f"Tournament ID {tournament_id} zaten mevcut, eklenmedi.")

    def season_data(self):
        verim = self.fetch_data()
        if verim is None:
            raise ValueError("Veri işlenmedi. Önce 'process_data()' metodunu çalıştırın.")
        
        verim = verim.drop_duplicates("season_id")
        
        season_cols = ["season_id", "season_name", "season_year","tournament_uniqueTournament_id"]

        for col in season_cols:
            if col not in verim.columns:
                verim.loc[:, col] = 0

        verim = verim[season_cols]

        for _, row in verim.iterrows():
            season_id = row['season_id']
            tournament_id = row.get('tournament_uniqueTournament_id')
    
            if pd.isna(season_id) or pd.isna(tournament_id):
                print(f"Geçersiz season_id veya tournament_id: Season ID={season_id}, Tournament ID={tournament_id}. Atlanıyor.")
                continue
    
            if not Season.objects.filter(season_id=season_id).exists():
                tournament_instance = Tournament.objects.filter(tournament_uniqueTournament_id=tournament_id).first()
                
                if tournament_instance:
                    Season.objects.create(
                        season_id=row['season_id'],
                        season_name=row['season_name'],
                        season_year=row['season_year'],
                        tournament=tournament_instance
                    )
                    print(f"Season ID {season_id} başarıyla eklendi.")
                else:
                    print(f"Tournament ID {tournament_id} bulunamadı, Season ID {season_id} eklenemedi.")
            else:
                print(f"Season ID {season_id} zaten mevcut, eklenmedi.")
    
    def team_data(self):

        data = self.fetch_data()
        homecols = [
                    "homeTeam_id", "homeTeam_name", "homeTeam_slug", "homeTeam_shortName",
                    "homeTeam_nameCode", "homeTeam_national","tournament_uniqueTournament_id","season_id"
                ]
        
        awaycols = [
                    "awayTeam_id", "awayTeam_name", "awayTeam_slug", "awayTeam_shortName",
                    "awayTeam_nameCode", "awayTeam_national","tournament_uniqueTournament_id","season_id"
                ]
        
        homedata = data[homecols]
        homedata.columns = ["Team_id", "Team_name", "Team_slug", "Team_shortName",
                                       "Team_nameCode", "Team_national","tournament_uniqueTournament_id","season_id"]
        awaydata = data[awaycols]
        awaydata.columns  = ["Team_id", "Team_name", "Team_slug", "Team_shortName",
                                       "Team_nameCode", "Team_national","tournament_uniqueTournament_id","season_id"]
        
        teamdata = pd.concat([homedata,awaydata])
        teamdata = teamdata.drop_duplicates(subset=["Team_id", "tournament_uniqueTournament_id", "season_id"])
       
        for _, row in teamdata.iterrows():
            team_id = row['Team_id']
            tournament_id = row['tournament_uniqueTournament_id']
            season_id = row['season_id']

            if pd.isna(team_id) or pd.isna(tournament_id) or pd.isna(season_id):
                print(f"Geçersiz değer: Team ID={team_id}, Tournament ID={tournament_id}, Season ID={season_id}. Atlanıyor.")
                continue

            tournament_instance = Tournament.objects.filter(tournament_uniqueTournament_id=tournament_id).first()
            season_instance = Season.objects.filter(season_id=season_id).first()

            if tournament_instance and season_instance:
                if not Team.objects.filter(team_id=team_id, tournament=tournament_instance, season=season_instance).exists():
                    Team.objects.create(
                        team_id=team_id,
                        team_name=row['Team_name'],
                        team_slug=row['Team_slug'],
                        team_shortName=row['Team_shortName'],
                        team_nameCode=row['Team_nameCode'],
                        team_national=row['Team_national'],
                        tournament=tournament_instance,
                        season=season_instance
                    )
                    print(f"Team ID {team_id} başarıyla eklendi.")
                else:
                    print(f"Team ID {team_id} zaten mevcut, eklenmedi.")
            else:
                print(f"Tournament ID {tournament_id} veya Season ID {season_id} bulunamadı.")

# SofascoreAPI sınıfını kullanarak team_data'yı veritabanına kaydetmek için:
api = SofascoreAPI()
teamdata = api.fetch_data()  # Veriyi getirin (API'den ya da dosyadan)
api.team_data()  # Veriyi işleyin ve veritabanına kaydedin


Team ID 3 başarıyla eklendi.
Team ID 44 başarıyla eklendi.
Team ID 14 başarıyla eklendi.
Team ID 2826 başarıyla eklendi.
Team ID 2836 başarıyla eklendi.
Team ID 2819 başarıyla eklendi.
Team ID 2817 başarıyla eklendi.
Team ID 2828 başarıyla eklendi.
Team ID 2689 başarıyla eklendi.
Team ID 2688 başarıyla eklendi.
Team ID 2719 başarıyla eklendi.
Team ID 2702 başarıyla eklendi.
Team ID 2701 başarıyla eklendi.
Team ID 2573 başarıyla eklendi.
Team ID 2524 başarıyla eklendi.
Team ID 1662 başarıyla eklendi.
Team ID 1646 başarıyla eklendi.
Team ID 1647 başarıyla eklendi.
Team ID 1681 başarıyla eklendi.
Team ID 1642 başarıyla eklendi.
Team ID 2951 başarıyla eklendi.
Team ID 2977 başarıyla eklendi.
Team ID 2971 başarıyla eklendi.
Team ID 1980 başarıyla eklendi.
Team ID 3050 başarıyla eklendi.
Team ID 6362 başarıyla eklendi.
Team ID 3053 başarıyla eklendi.
Team ID 202390 başarıyla eklendi.
Team ID 2918 başarıyla eklendi.
Team ID 2901 başarıyla eklendi.
Team ID 5044 başarıyla eklendi.
Team ID 3245 